In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/drw-crypto-market-prediction/sample_submission.csv
/kaggle/input/drw-crypto-market-prediction/train.parquet
/kaggle/input/drw-crypto-market-prediction/test.parquet


In [2]:
!rm -rf /kaggle/working/crypto_market_prediction
!git clone https://github.com/CapstoneTeam23UMICH/crypto_market_prediction.git

Cloning into 'crypto_market_prediction'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 67 (delta 22), reused 34 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (67/67), 9.78 MiB | 15.21 MiB/s, done.
Resolving deltas: 100% (22/22), done.


In [3]:
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sys.path.append('/kaggle/working/crypto_market_prediction')
from src.github_push_file import push_parquet_to_github
from src.get_refresh_metadata import get_feature_drift_df
from src.get_refresh_metadata import get_correlation_train_df
from src.get_refresh_metadata import get_correlation_test_df
from src.get_refresh_metadata import get_autocorrelation_train_df

In [4]:
df_drift = get_feature_drift_df()
df_corr_train = get_correlation_train_df()
df_autocorr = get_autocorrelation_train_df()

Loading existing df_feature_drift.parquet
Loading existing corr_long_train.parquet
Loading existing autocorr_long_train.parquet


In [5]:
train_path = '/kaggle/input/drw-crypto-market-prediction/train.parquet'
test_path = '/kaggle/input/drw-crypto-market-prediction/test.parquet'

df_train = pd.read_parquet(train_path).astype('float32')
df_test = pd.read_parquet(test_path).astype('float32')

In [9]:
known_features = ['bid_qty', 'ask_qty', 'buy_qty', 'sell_qty', 'volume']
target = 'label'
anonymized_features = sorted(list(set(df_train.columns) - set(known_features) - set([target])), key=lambda x: int(x[1:]))


In [ ]:





def preprocess_and_create_features(df):
    correlated_drop_features = set(df_corr_train[df_corr_train['corr'] > 0.99].y)

    for col in known_features:
        if col in data.columns:
            p1, p99 = data[col].quantile([0.01, 0.99])
            data[col] = data[col].clip(p1, p99)
            result_data[f'log_{col}'] = np.log1p(data[col].clip(lower=0)).astype('float32')
            result_data[f'{col}_sq'] = (data[col] ** 2).astype('float32')
            result_data[f'{col}_cube'] = (data[col] ** 3).astype('float32')
            result_data[f'{col}_sin'] = np.sin(data[col]).astype('float32')
            result_data[f'{col}_exp'] = np.exp(data[col].clip(upper=10)).astype('float32')
    
    result_data['buy_sell_ratio'] = (data['buy_qty'] / (data['sell_qty'] + 1e-6)).astype('float32')
    result_data['net_flow'] = (data['buy_qty'] - data['sell_qty']).astype('float32')
    result_data['order_imbalance'] = ((data['buy_qty'] - data['sell_qty']) / (data['buy_qty'] + data['sell_qty'] + 1e-6)).astype('float32')
    result_data['volume_imbalance'] = (data['volume'] * result_data['order_imbalance']).astype('float32')